In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input

import tensorflow as tf
with tf.device("/cpu:0"):
    seq_model = Sequential()
    seq_model.add(Dense(32, activation="relu", input_shape=(64, )))
    seq_model.add(Dense(32, activation="relu"))
    seq_model.add(Dense(10, activation="softmax"))

    input_tensor = Input(shape=(64, ))
    x = Dense(32, activation="relu")(input_tensor)
    x = Dense(32, activation="relu")(x)
    output_tensor = Dense(10, activation="softmax")(x)

    model = Model(input_tensor, output_tensor)
    model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
with tf.device("/cpu:0"):
    seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
import numpy as np

with tf.device("/cpu:0"):
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
    
    x_train = np.random.random((1000, 64))
    y_train = np.random.random((1000, 10))
    
    model.fit(x_train, y_train, epochs=10, batch_size=128)
    model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 1s 1ms/step - loss: 11.8099
Epoch 2/10
8/8 [==============================] - 0s 1ms/step - loss: 11.9714
Epoch 3/10
8/8 [==============================] - 0s 1ms/step - loss: 12.7313
Epoch 4/10
8/8 [==============================] - 0s 1ms/step - loss: 14.1545
Epoch 5/10
8/8 [==============================] - 0s 1ms/step - loss: 16.0890
Epoch 6/10
8/8 [==============================] - 0s 1ms/step - loss: 18.0590
Epoch 7/10
8/8 [==============================] - 0s 1ms/step - loss: 20.5751
Epoch 8/10
8/8 [==============================] - 0s 999us/step - loss: 23.7546
Epoch 9/10
8/8 [==============================] - 0s 1ms/step - loss: 27.2625
Epoch 10/10
32/32 [==============================] - 0s 645us/step - loss: 34.8427


In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, concatenate
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

with tf.device("/cpu:0"):
    text_input = Input(shape=(None, ), dtype="int32", name="text")
    embedded_text = Embedding(text_vocabulary_size, 64)(text_input)
    encoded_text = LSTM(32)(embedded_text)

    question_input = Input(shape=(None, ), dtype="int32", name="question")
    embedded_question = Embedding(question_vocabulary_size, 32)(question_input)
    encoded_question = LSTM(16)(embedded_question)

    concatenated = concatenate([encoded_text, encoded_question], axis=-1)

    answer = Dense(answer_vocabulary_size, activation="softmax")(concatenated)

    model = Model([text_input, question_input], answer)
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
    model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [11]:
import numpy as np
num_samples = 1000
max_length = 100

with tf.device("/cpu:0"):
    text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
    question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

    answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
    indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
    for i, x in enumerate(answers):
        x[indices[i]] = 1

    model.fit([text, question], answers, epochs=10, batch_size=128)
#     model.fit({"text": text, "question": question}, answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 1s 78ms/step - loss: 6.2944 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 1s 84ms/step - loss: 6.1734 - acc: 0.0030
Epoch 3/10
8/8 [==============================] - 1s 84ms/step - loss: 6.0996 - acc: 0.0030
Epoch 4/10
8/8 [==============================] - 1s 84ms/step - loss: 6.0178 - acc: 0.0050
Epoch 5/10
8/8 [==============================] - 1s 83ms/step - loss: 5.9151 - acc: 0.0070
Epoch 6/10
8/8 [==============================] - 1s 84ms/step - loss: 5.8267 - acc: 0.0160
Epoch 7/10
8/8 [==============================] - 1s 95ms/step - loss: 5.7550 - acc: 0.0210
Epoch 8/10
8/8 [==============================] - 1s 78ms/step - loss: 5.6762 - acc: 0.0230
Epoch 9/10
8/8 [==============================] - 1s 78ms/step - loss: 5.6048 - acc: 0.0250
Epoch 10/10
8/8 [==============================] - 1s 79ms/step - loss: 5.5509 - acc: 0.0220


In [13]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Embedding
from tensorflow.keras import Input
from tensorflow.keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

with tf.device("/cpu:0"):
    posts_input = Input(shape=(None, ), dtype="int32", name="posts")
    embedded_posts = Embedding(256, vocabulary_size)(posts_input)
    x = Conv1D(128, 5, activation="relu")(embedded_posts)
    x = MaxPooling1D(5)(x)
    x = Conv1D(256, 5, activation="relu")(x)
    x = Conv1D(256, 5, activation="relu")(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(256, 5, activation="relu")(x)
    x = Conv1D(256, 5, activation="relu")(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation="relu")(x)

    age_prediction = Dense(1, name="age")(x)
    income_prediction = Dense(num_income_groups, activation="softmax", name="income")(x)
    gender_prediction = Dense(1, activation="sigmoid", name="gender")(x)

    model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [ ]:
with tf.device("/cpu:0"):
    model.compile(optimizer="rmsprop", loss=["mse", "categorical_crossentropy", "binary_crossentropy"])
    model.compile(optimizer="rmsprop", loss={"age": "mse", "income": "categorical_crossentropy", "gender": "binary_crossentropy"})
    
    model.compile(optimizer="rmsprop", loss=["mse", "categorical_crossentropy", "binary_crossentropy"], loss_weights=[0.25, 1., 10.])
    model.compile(optimizer="rmsprop", loss={"age": "mse", "income": "categorical_crossentropy", "gender": "binary_crossentropy"},
                 loss_weights={"age": 0.25, "income": 1., "gender": 10.})
    
    model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)
    model.fit(posts, {"age": age_targets, "income": income_targets, "gender": gender_targets}, epochs=10, batch_size=64)

In [ ]:
from tensorflow.keras.layers import Conv2D, AveragePooling2D, concatenate

with tf.device("/cpu:0"):
    branch_a = Conv2D(128, 1, activation="relu", strides=2)(x)

    branch_b = Conv2D(128, 1, activation="relu")(x)
    branch_b = Conv2D(128, 3, activation="relu", strides=2)(branch_b)

    branch_c = AveragePooling2D(3, strides=2)(x)
    branch_c = Conv2D(128, 3, activation="relu")(branch_c)

    branch_d = Conv2D(128, 1, activation="relu")(x)
    branch_d = Conv2D(128, 3, activation="relu")(branch_d)
    branch_d = Conv2D(128, 3, activation="relu", strides=2)(branch_d)

    output= concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

In [15]:
from tensorflow.keras import layers

x = ...
y = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
y = layers.Conv2D(128, 3, activation="relu", padding="same")(y)
y = layers.Conv2D(128, 3, activation="relu", padding="same")(y)
y = layers.add([y, x])

x = ...
y = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
y = layers.Conv2D(128, 3, activation="relu", padding="same")(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding="same")(x)

y = layers.add([y, residual])

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

with tf.device("/cpu:0"):
    lstm = layers.LSTM(32)

    left_input = Input(shape=(None, 128))
    left_output = lstm(left_input)

    right_input = Input(shape=(None, 128))
    right_output = lstm(left_input)

    merged = layers.concatenate([left_output, right_output], axis=-1)
    predictions = layers.Dense(1, activation="sigmoid")(merge)

    model = Model([left_input, right_input], predictions)
    model.fit([left_data, right_data], targets)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import applications
from tensorflow.keras import Input

xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(letf_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)

In [ ]:
callbacks_list = {
    keras.callbacks.EarlyStopping(
        monitor="val_acc",
        patience=1,
    ),
    keras.callbacks.ModelCheckPoint(
        filepath="my_model.h5",
        monitor="val_loss",
        save_best_only=True,
    )
}

In [ ]:
import tensorflow.keras
import numpy as np

class ActivationLogger(keras.callbaks):
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError("Requires validation_data.")
            
            validation_sample = self.validation_data[0][0:1]
            activations = self.activations_model.predict(validation_sample)
            
            f = open("activation_at_epoch_" + str(epoch) + ".npz", "w")
            np.save(f, activations)
            f.close()

In [17]:
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 2000
max_len = 500

import tensorflow as tf
with tf.device("/cpu:0"):
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    x_train = pad_sequences(x_train)
    x_test = pad_sequences(x_test)

    model = keras.models.Sequential()
    model.add(layers.Embedding(max_features, 128, input_length=max_len, name="embed"))

    model.add(layers.Conv1D(32, 7, activation="relu"))
    model.add(layers.MaxPooling1D(5))
    model.add(layers.Conv1D(32, 7, activation="relu"))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(1))
    model.summary()
    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
________________________________________________

In [19]:
with tf.device("/cpu:0"):
    callbacks = [
        keras.callbacks.TensorBoard(
            log_dir="my_log_dir", histogram_freq=1, embeddings_freq=1
        )
    ]
    
    history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

In [24]:
from tensorflow.keras.utils import plot_model

with tf.device("/cpu:0"):
    plot_model(model, to_file="model.png")
    plot_model(model, show_shapes=True, to_file="model_shape.png")

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import 

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(SeparableConv2D(32, 3, activation="relu", input_shape=(height, width, channels, )))
model.add(SeparableConv2D(64, 3, activation="relu"))
model.add(MaxPooling2D(2))

model.add(SeparableConv2D(64, 3, activation="relu"))
model.add(SeparableConv2D(128, 3, activation="relu"))
model.add(MaxPooling2D(2))

model.add(SeparableConv2D(64, 3, activation="relu"))
model.add(SeparableConv2D(128, 3, activation="relu"))
model.add(GlobalAveragePooling2D())

model.add(Dense(32, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")